#1) IMPORTO LIBRERIAS

Primero se importan las librerias y se instala lo necesario

In [1]:
import pandas as pd
import io

import ast
import json
import datetime
import requests
import numpy as np

# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

!pip install -q findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

#2) CREO FUNCIONES

Se crean funciones de 3 tipos:

*   Auxiliares: Serán usadas como apoyo por las funciones para cargar bases y responder las preguntas
*   Cargar Bases: Serán usadas para transformar los json del s3 a tablas
*   Responder Preguntas: Serán usadas para responder preguntas del test






## AUXILIARES

In [2]:
# Formatear campo date de orders a week ISO
def FormatearFechaWeek(x):

  timestamp = datetime.datetime.fromtimestamp(x)
  timestampfin = timestamp.strftime('%Y-%W-%w')
  return timestampfin


#extraer el nombre del archivo json de la url
def ExtraerNombreJson(url):

  a = len(url)
  b = url[51:a]

  return b


#convertir de json a spark
def JsonToSpark(url):

  archivojson = ExtraerNombreJson(url)
  sp = spark.read.json(archivojson)
  return sp


#convertir de json a spark y luego a dataframe
def JsonToSparkToDataframe(url):

  # DescargarJson(url)
  archivojson = ExtraerNombreJson(url)

  sp = spark.read.json(archivojson)

  df = sp.toPandas()

  return df


# Formatear campo date de orders a week ISO
def FormatearFechaWeek(x):

  timestamp = datetime.datetime.fromtimestamp(x)
  timestampfin = timestamp.strftime('%Y-%W-%w')
  return timestampfin



## CARGAR BASES

Aqui descargamos el json orders y la convertimos en tabla

In [3]:
def TransformarJsonOrders(url):
  
  #obtenemos la data
  respuesta = requests.get(url)  

  #la data se transforma en un string
  data_t1 = respuesta.text

  #el string se transforma en una lista
  data_t2 = data_t1.splitlines()

  total_t1=pd.DataFrame()

  for elemento in data_t2[0:-1]:

    #el elemento de la lista se transforma en json
    elementojs = json.loads(elemento)

    cont = 0
    for elementoprod in elementojs["product"]:
      cont = cont + 1

      for llave in elementoprod.keys():

        campo = "order_product" + "_" + str(cont) + "_" + llave
        elementojs[campo] = elementoprod[llave]

    #el json se transforma en un dataframe
    elementodf = pd.json_normalize(elementojs)

    #se concatena con el resto de dataframes
    total_t1 = pd.concat([total_t1, elementodf],ignore_index=True)

    total_t1["date_formateada"] = total_t1['date'].apply(FormatearFechaWeek)

  return total_t1



Aqui descargamos el json sellers y la convertimos en tabla

In [4]:
def TransformarJsonSellers(url):
  !wget --continue 'https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json'
  total_t1 = JsonToSparkToDataframe(url)

  return total_t1

Aqui descargamos el json buyers y la convertimos en tabla

In [5]:
def TransformarJsonBuyers(url):

  #obtenemos la data
  respuesta = requests.get(url)

  #la data se transforma en un string
  data_t1 = respuesta.text

  #el string se transforma en una lista
  data_t2 = data_t1.splitlines()

  total_t1=pd.DataFrame()

  for elemento in data_t2:

    #elimino el ruido que esta como puntos en el phone
    elemento = elemento.replace(".", "")

    #el elemento de la lista se transforma en json
    elementojs = json.loads(elemento)

    #el json se transforma en un dataframe
    elementodf = pd.json_normalize(elementojs)

    #se concatena con el resto de dataframes
    total_t1 = pd.concat([total_t1, elementodf],ignore_index=True)
    # print(total_t1)

    # total_t1["start_formateada"] = total_t1['start'].apply(FormatearFecha)

  return total_t1

Aqui descargamos el json products y la convertimos en tabla

In [6]:
def TransformarJsonProducts(url):
  !wget --continue 'https://favo-data-test.s3.amazonaws.com/TestFolder/products.json'
  total_t1 = JsonToSparkToDataframe(url)

  return total_t1

Usamos una funcion para invocar las anteriores

In [7]:
def TransformarJsonADataframe(listaurl):

  df_orders = TransformarJsonOrders(listaurl[0])
  # print(df_orders)

  df_sellers = TransformarJsonSellers(listaurl[1])
  # print(df_sellers)

  df_buyers = TransformarJsonBuyers(listaurl[2])
  # print(df_buyers)

  df_products = TransformarJsonProducts(listaurl[3])
  # print(df_products)

  return df_orders,df_sellers,df_buyers,df_products  



## RESPONDER PREGUNTAS

##PREGUNTA 1

Respondemos la Pregunta 1 con un 1er Metodo que consiste en cruzar todas las tablas (orders,buyers,sellers,products) con python 

In [8]:
def Pregunta1Metodo1(df1,df2,df3,df4):

  #haciendo copias
  df1_t1 = df1.copy()
  df2_t1 = df2.copy()
  df3_t1 = df3.copy()
  df4_t1 = df4.copy()

  #renombramos y eliminamos campos de orders
  df1_t1.rename(columns={'date_formateada':'order_date_formateada'}, inplace=True)
  df1_t1.drop(['product','date'], axis = 1,inplace =True)

  #cruzamos orders con sellers
  total_t1 = pd.merge(df1_t1,df2_t1,on = 'seller_id',how = 'left')
  total_t1.rename(columns={'seller':'seller_name','phone':'seller_phone','start_formateada':'seller_start_formateada'}, inplace=True)
  total_t1.drop(['start'], axis = 1,inplace =True)

  #resultado cruzamos con buyers
  total_t2 = pd.merge(total_t1,df3_t1,on = 'buyer_id',how = 'left')
  total_t2.rename(columns={'buyer':'buyer_name','phone':'buyer_phone','start_formateada':'buyer_start_formateada'}, inplace=True)
  total_t2.drop(['start'], axis = 1,inplace =True)

  #resultado cruzamos con productos

  #para el producto1
  df4_t2 = df4_t1.copy()
  df4_t2.rename(columns={'name':'order_product_1_sku_name','sku':'order_product_1_sku','id':'order_product_1_id','price':'order_product_1_price'}, inplace=True)
  total_t3 = pd.merge(total_t2,df4_t2,on = 'order_product_1_sku',how = 'left')
  # total_t3.rename(columns={'phone':'buyer_phone','start_formateada':'buyer_start_formateada'}, inplace=True)
  # total_t3.drop(['start'], axis = 1,inplace =True)

  #para el producto2
  df4_t3 = df4_t1.copy()
  df4_t3.rename(columns={'name':'order_product_2_sku_name','sku':'order_product_2_sku','id':'order_product_2_id','price':'order_product_2_price'}, inplace=True)
  total_t4 = pd.merge(total_t3,df4_t3,on = 'order_product_2_sku',how = 'left')
  # total_t3.rename(columns={'phone':'buyer_phone','start_formateada':'buyer_start_formateada'}, inplace=True)
  # total_t3.drop(['start'], axis = 1,inplace =True)

  return total_t4


Respondemos la Pregunta 1 con un 2do Metodo que consiste en cruzar todas las tablas (orders,buyers,sellers,products) con spark sql 

In [9]:

def Pregunta1Metodo2(df1,df2,df3,df4):

  #haciendo copias
  df1_t1 = df1.copy()
  df2_t1 = df2.copy()
  df3_t1 = df3.copy()
  df4_t1 = df4.copy()

  df1_t2 = df1_t1[df1_t1['buyer_id'].notnull()]  
  sp1 =spark.createDataFrame(df1_t2)
  sp1.createOrReplaceTempView("sql_view_orders")

  sp2 =spark.createDataFrame(df2_t1)
  sp2.createOrReplaceTempView("sql_view_sellers")

  sp3 =spark.createDataFrame(df3_t1)
  sp3.createOrReplaceTempView("sql_view_buyers")

  sp4 =spark.createDataFrame(df4_t1)
  sp4.createOrReplaceTempView("sql_view_products")

  spark.sql("select tord.*, \
            tsel.*, \
            tbuy.*,\
            tpro1.*,\
            tpro2.*\
             FROM sql_view_orders tord\
        LEFT JOIN sql_view_sellers tsel ON  tord.seller_id = tsel.seller_id\
        LEFT JOIN sql_view_buyers tbuy ON tord.buyer_id = tbuy.buyer_id\
        LEFT JOIN sql_view_products tpro1 ON tord.order_product_1_sku = tpro1.sku\
        LEFT JOIN sql_view_products tpro2 ON tord.order_product_2_sku = tpro2.sku\
  ").show()

  return

Usamos una funcion Pregunta 1 para invocar los 2 metodos

In [10]:
def Pregunta1(df1,df2,df3,df4):


  p1m1 = Pregunta1Metodo1(df1,df2,df3,df4)  

  Pregunta1Metodo2(df1,df2,df3,df4)  

  return p1m1  

## PREGUNTA 2

Lo que se hizo aqui fue partir de la tabla ya con todos los cruces, luego calcular la venta por fila, ordenar por seller_id ,fecha (iso week) y cantidad vendida ascendentemente. Finalmente se agregó el campo crecimiento 

In [11]:
def Pregunta2(df1,df2,df3,df4):


  p1_t1 = Pregunta1Metodo1(df1,df2,df3,df4)
  # print(p1_t1)

  #calculo venta total por fila  
  p1_t1['order_product_1_venta'] = p1_t1['order_product_1_price']*p1_t1['order_product_1_qt']
  p1_t1['order_product_2_venta'] = p1_t1['order_product_2_price']*p1_t1['order_product_2_qt']
  p1_t1['order_total_venta'] = p1_t1['order_product_1_venta'] + p1_t1['order_product_2_venta']
  
  #selecciono algunos campos
  p1_t1 = p1_t1[['seller_id','order_date_formateada','order_total_venta']]

  #ordeno ascendentemente
  p1_t1.sort_values(by=['seller_id','order_date_formateada'],inplace = True)

  #calculo el crecimiento
  p1_t1['crecimiento'] = np.where(p1_t1['seller_id']==p1_t1.seller_id.shift(1), p1_t1['order_total_venta'] - p1_t1.order_total_venta.shift(1), 0)

  return p1_t1


##PREGUNTA 3

Lo que se hizo aqui fue partir de la tabla ya con todos los cruces, luego calcular la venta por fila, agrupar por seller_id y calcular el porcentaje de ventas de ese seller_id sobre el total

In [12]:
def Pregunta3(df1,df2,df3,df4):

  p1_t1 = Pregunta1Metodo1(df1,df2,df3,df4)

  p1_t1['order_product_1_venta'] = p1_t1['order_product_1_price']*p1_t1['order_product_1_qt']
  p1_t1['order_product_2_venta'] = p1_t1['order_product_2_price']*p1_t1['order_product_2_qt']
  p1_t1['order_total_venta'] = p1_t1['order_product_1_venta'] + p1_t1['order_product_2_venta']
  # Total = df['1'].sum()

  # tren_t3 = tren_t2.groupby(["Per_Salida_Date"], as_index=False)["Codigo"].count()
  p1_t2 = p1_t1.groupby(["seller_id"], as_index=False)["order_total_venta"].sum()
  totalventas = p1_t2['order_total_venta'].sum()
  p1_t2['total_ventas(%)'] = (p1_t2['order_total_venta']/totalventas) * 100

  p1_t2 = p1_t2[['seller_id','total_ventas(%)']]

  return p1_t2



## PREGUNTA 4

Para completar el buyer_id a las ordenes que faltaban se hizo lo siguiente:


1.   Se concateno el sk del producto 1 y producto 2 (sk1+sk2) y se agrupo contando la cantidad de buyer_id y en la muy gran mayoria se vio que para cada sk1+sk2 solo habia un unico buyer_id entonces no habia mucha sentido hacer imputacion con sk1+sk2 y se decidio hacer por el sk1
2.   Ubicamos los sk1 donde esten buyer_id nulos y se imputa con los datos correspondientes de sk1 (buyer_id,buyer_name y buyer_phone)



In [13]:

def Pregunta4(df1,df2,df3,df4):
  p1_t1 = Pregunta1Metodo1(df1,df2,df3,df4)

  p1_t1["sk1+sk2"] = p1_t1["order_product_1_sku"].apply(str) + p1_t1["order_product_2_sku"].apply(str)
  # p1_t1.info()

  p1_t1["Cantidad"] = p1_t1["buyer_id"]   

  #la gran mayoria combinacion de los 2 productos comprados es unica
  p1_t2 = p1_t1.groupby(["sk1+sk2"], as_index=False)["Cantidad"].count()


  p1_t2 = p1_t1.groupby(["order_product_1_sku","buyer_id","buyer_name","buyer_phone"], as_index=False)["Cantidad"].count()

  #ubicamos los order_product_1_sku donde esten buyer_id nulos
  listas_order_product_1_sku_nulos = p1_t1.loc[p1_t1["buyer_id"].isnull(),"order_product_1_sku"].unique()
  
  #recorremos cada order_product_1_sku que contiene buyer_id nulo y cada buyer_id de un order_product_1_sku se imputa con el 1er valor de la lista de buyers de ese order_product_1_sku
  for l in listas_order_product_1_sku_nulos:

    p1_t1.loc[(p1_t1["buyer_id"].isnull()) & (p1_t1["order_product_1_sku"] == l),"buyer_id"] = p1_t2.loc[p1_t2["order_product_1_sku"] == l,"buyer_id"].values[0]
    p1_t1.loc[(p1_t1["buyer_name"].isnull()) & (p1_t1["order_product_1_sku"] == l),"buyer_name"] = p1_t2.loc[p1_t2["order_product_1_sku"] == l,"buyer_name"].values[0]
    p1_t1.loc[(p1_t1["buyer_phone"].isnull()) & (p1_t1["order_product_1_sku"] == l),"buyer_phone"] = p1_t2.loc[p1_t2["order_product_1_sku"] == l,"buyer_phone"].values[0]
    # p1_t1.loc[(p1_t1["buyer_start_formateada"].isnull()) & (p1_t1["order_product_1_sku"] == l),"buyer_start_formateada"] = p1_t2.loc[p1_t2["order_product_1_sku"] == l,"buyer_start_formateada"].values[0]

  # p1_t4 = p1_t1[p1_t1['buyer_id'].isnull()]

  del p1_t1["Cantidad"]
  p1_t1.info()


  return p1_t1



Se tiene una funcion que invoca las respuestas a las preguntas y las descarga en excel

In [14]:

def ResponderPreguntas(dfo,dfs,dfb,dfp):

  dfo_t1 = dfo.copy()
  dfs_t1 = dfs.copy()
  dfb_t1 = dfb.copy()
  dfp_t1 = dfp.copy()

  print( "############################################## SOLUCION PREGUNTA 1 (2 METODOS)##############################################")
  p1 = Pregunta1(dfo_t1,dfs_t1,dfb_t1,dfp_t1)
  print(p1)
  p1.to_excel("SOLUCION_PREGUNTA_1.xlsx") 
  from google.colab import files
  files.download("SOLUCION_PREGUNTA_1.xlsx") 


  print( "############################################## SOLUCION PREGUNTA 2 ##############################################")
  p2 = Pregunta2(dfo_t1,dfs_t1,dfb_t1,dfp_t1)
  print(p2)
  p2.to_excel("SOLUCION_PREGUNTA_2.xlsx") 
  from google.colab import files
  files.download("SOLUCION_PREGUNTA_2.xlsx")

  print( "############################################## SOLUCION PREGUNTA 3 ##############################################")
  p3 = Pregunta3(dfo_t1,dfs_t1,dfb_t1,dfp_t1)
  print(p3)
  p3.to_excel("SOLUCION_PREGUNTA_3.xlsx") 
  from google.colab import files
  files.download("SOLUCION_PREGUNTA_3.xlsx")

  print( "############################################## SOLUCION PREGUNTA 4 ##############################################")
  p4 = Pregunta4(dfo_t1,dfs_t1,dfb_t1,dfp_t1)
  print(p4)
  p1.to_excel("SOLUCION_PREGUNTA_4.xlsx") 
  from google.colab import files
  files.download("SOLUCION_PREGUNTA_4.xlsx")

  return p1,p2,p3,p4


# 3) MAIN

Luego de haber definido todas las funciones necesarias en el punto anterior en este main ejecutamos todo el proceso de la siguiente forma:


1.   Defino mi lista de urls (orders,sellers,buyers y products)
2.   Transformo los json de las urls a tablas
3.   Usando las tablas del punto anterior respondo todas las preguntas del test


In [15]:

def main():
  
  #URLS S3
  listaurl = ['https://favo-data-test.s3.amazonaws.com/TestFolder/orders.json','https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json','https://favo-data-test.s3.amazonaws.com/TestFolder/buyers.json','https://favo-data-test.s3.amazonaws.com/TestFolder/products.json']
 
  #TRANSFORMO JSONS A TABLAS 
  df_orders,df_sellers,df_buyers,df_products = TransformarJsonADataframe(listaurl)

  #USO ESOS DATAFRAMES PARA RESPONDER LAS PREGUNTAS
  p1_respondida,p2_respondida,p3_respondida,p4_respondida = ResponderPreguntas(df_orders,df_sellers,df_buyers,df_products)   


  return p1_respondida,p2_respondida,p3_respondida,p4_respondida      
     

In [16]:
main()

--2021-09-17 10:28:26--  https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json
Resolving favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)... 52.216.18.184
Connecting to favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)|52.216.18.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10397 (10K) [application/json]
Saving to: ‘sellers.json’

sellers.json        100%[===================>]  10.15K  --.-KB/s    in 0s      

2021-09-17 10:28:26 (81.3 MB/s) - ‘sellers.json’ saved [10397/10397]

--2021-09-17 10:28:33--  https://favo-data-test.s3.amazonaws.com/TestFolder/products.json
Resolving favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)... 52.217.196.153
Connecting to favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)|52.217.196.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9133 (8.9K) [application/json]
Saving to: ‘products.json’

products.json       100%[====

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

############################################## SOLUCION PREGUNTA 2 ##############################################
                                 seller_id  ... crecimiento
12   0158E0BA-7916-434F-AEAE-1C30-DF768884  ...        0.00
191  0158E0BA-7916-434F-AEAE-1C30-DF768884  ...      -10.71
199  0195516D-AD20-4FAF-90E6-1C47-DAFAAA73  ...        0.00
184  0195516D-AD20-4FAF-90E6-1C47-DAFAAA73  ...       13.39
87   0195516D-AD20-4FAF-90E6-1C47-DAFAAA73  ...       61.68
..                                     ...  ...         ...
44   FA1A3B8F-9273-4CE2-95ED-D508-200F740C  ...       69.74
47   FA1A3B8F-9273-4CE2-95ED-D508-200F740C  ...       10.99
138  FBE555F8-864A-4118-BFFD-C432-4054ED52  ...        0.00
84   FCB585EF-A4D5-437D-951F-EC84-5171FFC6  ...        0.00
58   FE1085CE-EF48-4B89-8B06-14CB-D0E5F6C1  ...        0.00

[200 rows x 4 columns]


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

############################################## SOLUCION PREGUNTA 4 ##############################################
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   order_id                  200 non-null    object 
 1   buyer_id                  200 non-null    object 
 2   seller_id                 200 non-null    object 
 3   order_product_1_sku       200 non-null    int64  
 4   order_product_1_qt        200 non-null    int64  
 5   order_product_2_sku       200 non-null    int64  
 6   order_product_2_qt        200 non-null    int64  
 7   order_date_formateada     200 non-null    object 
 8   seller_phone              200 non-null    int64  
 9   seller_name               200 non-null    object 
 10  buyer_name                200 non-null    object 
 11  buyer_phone               200 non-null    float64
 12  order_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(                                  order_id  ... order_product_2_price
 0    8275800A-F1C9-49BD-97E1-18DA-64C3A758  ...                  8.84
 1    341DDEC4-8D7D-4C4C-B8E8-6EA8-F5A8692B  ...                  5.40
 2    B74FEE59-0E7A-4B06-8A05-C56F-5E6A7903  ...                 10.38
 3    93924557-72E7-4DAA-83FF-613B-363FE64C  ...                  3.66
 4    42654D33-759A-4F3D-ABAC-DB89-68BC6B24  ...                  8.35
 ..                                     ...  ...                   ...
 195  BDB9D2E3-2E49-455A-924E-9F3F-98E1DFED  ...                  4.68
 196  3CDE425C-3130-4192-9D44-E16B-C9E5F45A  ...                  6.63
 197  9966E05B-4AAA-4D2F-8B62-97E6-FCA166B8  ...                 11.52
 198  B80537D7-56F4-42ED-A665-54EE-65AA4599  ...                  7.35
 199  9BF5274A-743C-4B04-858F-CCC5-48C3460B  ...                  4.73
 
 [200 rows x 18 columns],
                                  seller_id  ... crecimiento
 12   0158E0BA-7916-434F-AEAE-1C30-DF768884  ...        0.0

# 4) PRUEBAS UNITARIAS

En este punto se hicieron pruebas de la cantidad de filas para las 4 tablas y se puso a proposito distintas filas a las reales de las buyers y orders para ver el 'Failed' 

In [17]:
import unittest

listaurl = ['https://favo-data-test.s3.amazonaws.com/TestFolder/orders.json','https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json','https://favo-data-test.s3.amazonaws.com/TestFolder/buyers.json','https://favo-data-test.s3.amazonaws.com/TestFolder/products.json']

class CalcularFilas:
    
    def __init__(self):
        pass

    def CalcularFilasOrders(self,x):
        data_orders = TransformarJsonOrders(x)
        return len(data_orders.index)

    def CalcularFilasSellers(self,x):
        data_sellers = TransformarJsonSellers(x)
        return len(data_sellers.index)

    def CalcularFilasBuyers(self,x):
        data_buyers = TransformarJsonBuyers(x)
        return len(data_buyers.index)

    def CalcularFilasOrders(self,x):
        data_buyers = TransformarJsonProducts(x)
        return len(data_buyers.index)


class TestCalcularFilas(unittest.TestCase):

    def test_CalcularFilasOrders(self):
        '''Test case function for CalcularFilasOrders'''
        self.calc = CalcularFilas()
        result = self.calc.CalcularFilasOrders(listaurl[0])
        expected = 120
        self.assertEqual(result, expected)

    def test_CalcularFilasSellers(self):
        '''Test case function for CalcularFilasSellers'''
        self.calc = CalcularFilas()
        result = self.calc.CalcularFilasSellers(listaurl[1])
        expected = 89
        self.assertEqual(result, expected)

    def test_CalcularFilasBuyers(self):
        '''Test case function for CalcularFilasBuyers'''
        self.calc = CalcularFilas()
        result = self.calc.CalcularFilasBuyers(listaurl[2])
        expected = 150
        self.assertEqual(result, expected)


    def test_CalcularFilasOrders(self):
        '''Test case function for CalcularFilasOrders'''
        self.calc = CalcularFilas()
        result = self.calc.CalcularFilasOrders(listaurl[3])
        expected = 200
        self.assertEqual(result, expected)


In [18]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_CalcularFilasBuyers (__main__.TestCalcularFilas)
Test case function for CalcularFilasBuyers ... FAIL
test_CalcularFilasOrders (__main__.TestCalcularFilas)
Test case function for CalcularFilasOrders ... 

--2021-09-17 10:28:53--  https://favo-data-test.s3.amazonaws.com/TestFolder/products.json
Resolving favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)... 52.216.230.19
Connecting to favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)|52.216.230.19|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



FAIL
test_CalcularFilasSellers (__main__.TestCalcularFilas)
Test case function for CalcularFilasSellers ... 

--2021-09-17 10:28:54--  https://favo-data-test.s3.amazonaws.com/TestFolder/sellers.json
Resolving favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)... 52.216.97.139
Connecting to favo-data-test.s3.amazonaws.com (favo-data-test.s3.amazonaws.com)|52.216.97.139|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



ok

FAIL: test_CalcularFilasBuyers (__main__.TestCalcularFilas)
Test case function for CalcularFilasBuyers
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-17-99c14596a8d2>", line 48, in test_CalcularFilasBuyers
    self.assertEqual(result, expected)
AssertionError: 200 != 150

FAIL: test_CalcularFilasOrders (__main__.TestCalcularFilas)
Test case function for CalcularFilasOrders
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-17-99c14596a8d2>", line 56, in test_CalcularFilasOrders
    self.assertEqual(result, expected)
AssertionError: 104 != 200

----------------------------------------------------------------------
Ran 3 tests in 1.650s

FAILED (failures=2)
